In [1]:
import csv
import pandas as pd
import requests
import os
from collections import defaultdict
from datetime import timedelta
import sklearn.linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import geopandas
import matplotlib

/home/gautam/.local/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
data=pd.read_csv("../dataset/pred_time.csv")

In [3]:
df=data[data["since_prev_break"]!=float('inf')]
df
#"break_date","InstallYear_2"

,Unnamed: 0,FacilityID,InstallYear,Diameter,break_date,CI,SPUN,SAND,DI,InstallYear_2,break_date_2,since_prev_break,prior_breaks
0,0,MAIN-6943-13636,1969,6.0,1997-01-02,0,0,0,1,1969-01-01,-8435.0,10228.0,0.0
1,1,MAIN-6336-18809,1959,8.0,1997-02-04,1,0,0,0,1959-01-01,-8402.0,13914.0,0.0
2,2,MAIN-5744-15604,1917,10.0,1997-10-08,0,0,1,0,1917-01-01,-8156.0,29500.0,0.0
3,3,MAIN-3749-6440,1950,6.0,1998-08-01,0,1,0,0,1950-01-01,-7859.0,17744.0,0.0
4,4,MAIN-6860-16497,1966,8.0,1998-08-21,1,0,0,0,1966-01-01,-7839.0,11920.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3400,3400,MAIN-2853-3313,1965,10.0,2019-09-25,1,0,0,0,1965-01-01,-134.0,19990.0,0.0
3401,3401,MAIN-4949-5931,2006,6.0,2019-09-28,0,0,0,1,2006-01-01,-131.0,5018.0,0.0
3402,3402,MAIN-2859-4488,1995,10.0,2019-10-02,0,0,0,1,1995-01-01,-127.0,2108.0,1.0
3403,3403,MAIN-6729-19170,1970,12.0,2019-10-06,0,0,0,1,1970-01-01,-123.0,18175.0,0.0


In [4]:
xcols = ["prior_breaks", "InstallYear","Diameter","CI","SPUN","SAND","DI","break_date_2"]
ycol = "since_prev_break"

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df[xcols], df[ycol], test_size=0.1)

In [6]:
from sklearn import linear_model
import numpy as np                   
    
clf=linear_model.Ridge(alpha=0.01)
clf.fit(X_train, y_train)
prediction=clf.predict(X_test)

prediction[prediction<0]=0

In [7]:
##Now retrain on the entire set and formulate predictions

In [8]:
clf2=linear_model.Ridge(alpha=0.01)
clf2.fit(df[xcols], df[ycol])
data=data.fillna(0)
prediction2=clf2.predict(data[xcols])

In [9]:
data=data.replace(float('inf'), np.nan)
data=data.fillna(0)
data["Predict"]=round(prediction2-data["since_prev_break"])/365
data[data["Predict"]<0]=0

In [10]:
wm = geopandas.read_file("zip://../dataset/WaterMains.zip")
wm

,OBJECTID,FacilityID,InstallYea,LifecycleS,Diameter,Material,Bagged,LiningType,Shape_STLe,geometry
0,1,MAIN-4869-1,1981,ACT,16.0,DI,0,CM,3.646795,"LINESTRING (818246.809 465305.287, 818250.456 ..."
1,2,MAIN-4869-2,1981,ACT,16.0,DI,0,CM,3.500123,"LINESTRING (818250.645 465308.776, 818250.456 ..."
2,3,MAIN-4969-3,1981,ACT,16.0,DI,0,CM,3.499850,"LINESTRING (819023.477 465921.541, 819025.375 ..."
3,4,MAIN-4969-4,1981,ACT,16.0,DI,0,CM,7.648740,"LINESTRING (819025.375 465918.600, 819032.423 ..."
4,5,MAIN-4969-5,1981,ACT,16.0,DI,0,CM,1127.874973,"LINESTRING (818330.485 465303.820, 818346.420 ..."
...,...,...,...,...,...,...,...,...,...,...
22515,419693,MAIN-1448-25169,2020,ACT,8.0,DI,1,CM,111.669489,"LINESTRING (773693.791 484574.175, 773693.674 ..."
22516,419694,MAIN-1447-25170,2020,ACT,8.0,DI,1,CM,831.525282,"LINESTRING (773691.327 484690.979, 773691.253 ..."
22517,420093,MAIN-2656-25171,1969,ACT,6.0,DI,1,CM,38.955988,"LINESTRING (788198.035 477702.644, 788198.153 ..."
22518,1168,MAIN-2455-1168,1994,ACT,16.0,DI,1,CM,516.630823,"LINESTRING (785556.369 477740.132, 785556.743 ..."


In [11]:
ms=wm.merge(data, on='FacilityID', how='left')
ms

,OBJECTID,FacilityID,InstallYea,LifecycleS,Diameter_x,Material,Bagged,LiningType,Shape_STLe,geometry,...,break_date,CI,SPUN,SAND,DI,InstallYear_2,break_date_2,since_prev_break,prior_breaks,Predict
0,1,MAIN-4869-1,1981,ACT,16.0,DI,0,CM,3.646795,"LINESTRING (818246.809 465305.287, 818250.456 ...",...,0,0.0,0.0,0.0,1.0,1981-01-01,0.0,0.0,0.0,33.657534
1,2,MAIN-4869-2,1981,ACT,16.0,DI,0,CM,3.500123,"LINESTRING (818250.645 465308.776, 818250.456 ...",...,0,0.0,0.0,0.0,1.0,1981-01-01,0.0,0.0,0.0,33.657534
2,3,MAIN-4969-3,1981,ACT,16.0,DI,0,CM,3.499850,"LINESTRING (819023.477 465921.541, 819025.375 ...",...,0,0.0,0.0,0.0,1.0,1981-01-01,0.0,0.0,0.0,33.657534
3,4,MAIN-4969-4,1981,ACT,16.0,DI,0,CM,7.648740,"LINESTRING (819025.375 465918.600, 819032.423 ...",...,0,0.0,0.0,0.0,1.0,1981-01-01,0.0,0.0,0.0,33.657534
4,5,MAIN-4969-5,1981,ACT,16.0,DI,0,CM,1127.874973,"LINESTRING (818330.485 465303.820, 818346.420 ...",...,0,0.0,0.0,0.0,1.0,1981-01-01,0.0,0.0,0.0,33.657534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23039,419693,MAIN-1448-25169,2020,ACT,8.0,DI,1,CM,111.669489,"LINESTRING (773693.791 484574.175, 773693.674 ...",...,0,0.0,0.0,0.0,1.0,2020-01-01,0.0,0.0,0.0,1.150685
23040,419694,MAIN-1447-25170,2020,ACT,8.0,DI,1,CM,831.525282,"LINESTRING (773691.327 484690.979, 773691.253 ...",...,0,0.0,0.0,0.0,1.0,2020-01-01,0.0,0.0,0.0,1.150685
23041,420093,MAIN-2656-25171,1969,ACT,6.0,DI,1,CM,38.955988,"LINESTRING (788198.035 477702.644, 788198.153 ...",...,0,0.0,0.0,0.0,1.0,1969-01-01,0.0,0.0,0.0,37.364384
23042,1168,MAIN-2455-1168,1994,ACT,16.0,DI,1,CM,516.630823,"LINESTRING (785556.369 477740.132, 785556.743 ...",...,0,0.0,0.0,0.0,1.0,1994-01-01,0.0,0.0,0.0,24.169863


In [12]:
mst = geopandas.read_file("zip://../dataset/Hi.zip")
mst

,ID,geometry
0,1,"POLYGON ((750000.000 520000.000, 751000.000 52..."
1,2,"POLYGON ((751000.000 520000.000, 752000.000 52..."
2,3,"POLYGON ((752000.000 520000.000, 753000.000 52..."
3,4,"POLYGON ((753000.000 520000.000, 754000.000 52..."
4,5,"POLYGON ((754000.000 520000.000, 755000.000 52..."
...,...,...
7695,7696,"POLYGON ((855000.000 451000.000, 856000.000 45..."
7696,7697,"POLYGON ((856000.000 451000.000, 857000.000 45..."
7697,7698,"POLYGON ((857000.000 451000.000, 858000.000 45..."
7698,7699,"POLYGON ((858000.000 451000.000, 859000.000 45..."


In [13]:
m = geopandas.sjoin(mst, ms, how="inner", op='intersects')
g=m[["geometry","Predict"]]
g["index"]=g.index
g

/home/gautam/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: PROJCS["NAD_1983_HARN_WISCRS_Dane_County_Feet",GEO ...

  """Entry point for launching an IPython kernel.
/home/gautam/.local/lib/python3.6/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


,geometry,Predict,index
198,"POLYGON ((838000.000 519000.000, 839000.000 51...",9.739726,198
198,"POLYGON ((838000.000 519000.000, 839000.000 51...",9.739726,198
308,"POLYGON ((838000.000 518000.000, 839000.000 51...",9.739726,308
308,"POLYGON ((838000.000 518000.000, 839000.000 51...",9.739726,308
418,"POLYGON ((838000.000 517000.000, 839000.000 51...",9.739726,418
...,...,...,...
6970,"POLYGON ((790000.000 457000.000, 791000.000 45...",16.309589,6970
6970,"POLYGON ((790000.000 457000.000, 791000.000 45...",17.038356,6970
6970,"POLYGON ((790000.000 457000.000, 791000.000 45...",15.019178,6970
6970,"POLYGON ((790000.000 457000.000, 791000.000 45...",15.019178,6970


In [14]:
g=g.groupby(by='index').agg({'Predict': 'min','geometry':'first'}).reset_index()
g=g[["geometry","Predict"]]
g=geopandas.GeoDataFrame(g)
g

,geometry,Predict
0,"POLYGON ((838000.000 519000.000, 839000.000 51...",9.739726
1,"POLYGON ((838000.000 518000.000, 839000.000 51...",3.901370
2,"POLYGON ((838000.000 517000.000, 839000.000 51...",9.739726
3,"POLYGON ((839000.000 517000.000, 840000.000 51...",13.389041
4,"POLYGON ((837000.000 516000.000, 838000.000 51...",4.632877
...,...,...
1984,"POLYGON ((787000.000 457000.000, 788000.000 45...",11.090411
1985,"POLYGON ((788000.000 457000.000, 789000.000 45...",9.630137
1986,"POLYGON ((789000.000 457000.000, 790000.000 45...",10.638356
1987,"POLYGON ((790000.000 457000.000, 791000.000 45...",2.019178


In [16]:
one = geopandas.read_file("../dataset/1yr_prediction")
one=one[["geometry","prediction"]]
one["Span"]=1
one["index"]=one.index
one=geopandas.GeoDataFrame(one)
one

,geometry,prediction,Span,index
0,"LINESTRING (839120.367 491947.125, 839128.216 ...",8.972000e-12,1,0
1,"LINESTRING (805899.264 468237.072, 806078.398 ...",1.131859e-03,1,1
2,"LINESTRING (817263.913 468292.796, 817376.537 ...",3.063407e-02,1,2
3,"LINESTRING (812586.155 468311.223, 812907.967 ...",5.913109e-05,1,3
4,"LINESTRING (806832.776 474127.089, 806866.785 ...",3.661825e-01,1,4
...,...,...,...,...
10216,"LINESTRING (782138.210 468960.585, 781406.573 ...",5.466866e-02,1,10216
10217,"LINESTRING (782148.645 469701.925, 782138.210 ...",5.466866e-02,1,10217
10218,"LINESTRING (782138.210 468960.585, 782150.621 ...",5.466866e-02,1,10218
10219,"LINESTRING (789389.038 465926.289, 789227.525 ...",5.466866e-02,1,10219


In [17]:
five = geopandas.read_file("../dataset/5yr_prediction")
five=five[["geometry","prediction"]]
five["Span"]=5
five["index"]=five.index
five=geopandas.GeoDataFrame(five)
five

,geometry,prediction,Span,index
0,"LINESTRING (839120.367 491947.125, 839128.216 ...",0.082593,5,0
1,"LINESTRING (805899.264 468237.072, 806078.398 ...",0.259088,5,1
2,"LINESTRING (817263.913 468292.796, 817376.537 ...",0.381232,5,2
3,"LINESTRING (812586.155 468311.223, 812907.967 ...",0.261788,5,3
4,"LINESTRING (806832.776 474127.089, 806866.785 ...",0.309893,5,4
...,...,...,...,...
10216,"LINESTRING (782138.210 468960.585, 781406.573 ...",0.068091,5,10216
10217,"LINESTRING (782148.645 469701.925, 782138.210 ...",0.068091,5,10217
10218,"LINESTRING (782138.210 468960.585, 782150.621 ...",0.068091,5,10218
10219,"LINESTRING (789389.038 465926.289, 789227.525 ...",0.068091,5,10219


In [18]:
ten = geopandas.read_file("../dataset/10yr_prediction")
ten=ten[["geometry","prediction"]]
ten["Span"]=10
ten["index"]=ten.index
ten=geopandas.GeoDataFrame(ten)
ten

,geometry,prediction,Span,index
0,"LINESTRING (839120.367 491947.125, 839128.216 ...",0.216477,10,0
1,"LINESTRING (805899.264 468237.072, 806078.398 ...",0.378009,10,1
2,"LINESTRING (817263.913 468292.796, 817376.537 ...",0.548636,10,2
3,"LINESTRING (812586.155 468311.223, 812907.967 ...",0.443005,10,3
4,"LINESTRING (806832.776 474127.089, 806866.785 ...",0.363717,10,4
...,...,...,...,...
10216,"LINESTRING (782138.210 468960.585, 781406.573 ...",0.091044,10,10216
10217,"LINESTRING (782148.645 469701.925, 782138.210 ...",0.091044,10,10217
10218,"LINESTRING (782138.210 468960.585, 782150.621 ...",0.091044,10,10218
10219,"LINESTRING (789389.038 465926.289, 789227.525 ...",0.091044,10,10219


In [19]:
twenty = geopandas.read_file("../dataset/20yr_prediction")
twenty=twenty[["geometry","prediction"]]
twenty["Span"]=20
twenty["index"]=twenty.index
twenty=geopandas.GeoDataFrame(twenty)
twenty

,geometry,prediction,Span,index
0,"LINESTRING (839120.367 491947.125, 839128.216 ...",0.601128,20,0
1,"LINESTRING (805899.264 468237.072, 806078.398 ...",0.956129,20,1
2,"LINESTRING (817263.913 468292.796, 817376.537 ...",0.978976,20,2
3,"LINESTRING (812586.155 468311.223, 812907.967 ...",0.946134,20,3
4,"LINESTRING (806832.776 474127.089, 806866.785 ...",0.930748,20,4
...,...,...,...,...
10216,"LINESTRING (782138.210 468960.585, 781406.573 ...",0.179740,20,10216
10217,"LINESTRING (782148.645 469701.925, 782138.210 ...",0.179740,20,10217
10218,"LINESTRING (782138.210 468960.585, 782150.621 ...",0.179740,20,10218
10219,"LINESTRING (789389.038 465926.289, 789227.525 ...",0.179740,20,10219


In [20]:
master=one.append(five)
master=master.append(ten)
master=master.append(twenty)
master

,geometry,prediction,Span,index
0,"LINESTRING (839120.367 491947.125, 839128.216 ...",8.972000e-12,1,0
1,"LINESTRING (805899.264 468237.072, 806078.398 ...",1.131859e-03,1,1
2,"LINESTRING (817263.913 468292.796, 817376.537 ...",3.063407e-02,1,2
3,"LINESTRING (812586.155 468311.223, 812907.967 ...",5.913109e-05,1,3
4,"LINESTRING (806832.776 474127.089, 806866.785 ...",3.661825e-01,1,4
...,...,...,...,...
10216,"LINESTRING (782138.210 468960.585, 781406.573 ...",1.797397e-01,20,10216
10217,"LINESTRING (782148.645 469701.925, 782138.210 ...",1.797397e-01,20,10217
10218,"LINESTRING (782138.210 468960.585, 782150.621 ...",1.797397e-01,20,10218
10219,"LINESTRING (789389.038 465926.289, 789227.525 ...",1.797397e-01,20,10219


In [25]:
master2=master.groupby(by='index').agg({'prediction': 'max',"geometry":'first'})
master2["span"]=0
master2=geopandas.GeoDataFrame(master2)

In [ ]:
matplotlib.rcParams.update({'font.size': 14})

fig,ax = plt.subplots(figsize=(10,10))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05,label="Risk")
g.plot(column="Predict",cmap="greys_r",ax=ax,legend=True,cax=cax,legend_kwds={'label': "Duration Until Next Break (yrs) "})
master2.plot(column="prediction",cmap="Reds",ax=ax)
ax.axis("off")
#plt.tight_layout()

#plt.savefig('riskmap.pdf',format='pdf')

In [22]:
for i in master2.index:
    if master2.at[i, "prediction"] == one.at[i, "prediction"] and master2.at[i, "index"] == one.at[i, "index"]:
        master2.iloc[i, "span"]= one.at[i, "Span"]